# Datos en Docker

## Bind mounts

Los contenedores son entidades autocontenidas que no pueden acceder por defecto al SO de la maquina que los esta ejecutando, de hecho ellos no saben que estan dentro de una maquina desde el punto de vista del contenedor.

Sin embargo muchas veces, es necesario guardar los datos del contenedor, para usarlos en otros procesos. 

Crearemos un nuevo folder llamado *myDockerData*

Para esta practica crearemos un container basado MongoDB:

    sudo docker run --name myMongoPlatzi --detach mongo:5.0

Hubiera podido colocar solo *mongo* al final pero como ya tenia una imagen guardada, para hacerlo mas rapido, baso mi nuevo conteneder en la existente.

Verifico este corriendo:

    sudo docker ps

Para generar algunso datos dentro del contenedor, primero executamos una terminal:

    sudo docker exec -it  myMongoPlatzi bash

El prompt cambia, indicando podemos ejecutar comandos dentro del contenedor:

    root@4e10d97111ad:/#

Este contenedor viene con un binario que es el cliente de la base de datos, 

    mongo

Tambien puedo usar *mongosh* como es constumbre en el curso de MongoDB

    mongosh


Y ahora, algunso comandos de Mongo para crear una base de datos, la coleccion y almenos un documento.

    use platzi
    db.user.insert({ name: "rodrigo", age: 30 })

Verificando este creado:

    db.user.insert({ name: "rodrigo", age: 30 })

Sin embargo, cuando paremos el contenedor, y cuando lo borremos, perderemos la informacion. 😞

### Creando el bind mount

Una forma de no perder la informacion, es tener un directorio en mi maquina que tenga una version espejada de lo que ocurre en el contenedor. Para hacer esto haremos un *bind mount*

Borremos el contenedor y creemos uno nuevo con el mismo nombre:

    sudo docker rm myMongoPlatzi


con *-v* o *--volume* le indicamos que queremos hacer un *bind mount*. Mongo guarda los datos en **/data/db**. Entonces iria asi 

**--volume (ruta local):(ruta a copiar)**

    docker run -d --name myMongoPlatzi --volume /home/carlos/Documents/curso_docker/myDockerData:/data/db mongo:5.0

De igual manera, volvemos a crear los datos nuevamente

    mongosh

    use platzi

    db.users.insertMany([{ name: "gustavo", age: 31}, { name: "rodrigo", age:19 }])

    db.users.find()

Salgamos, y borremos el contenedor totalmente

    sudo docker rm -f myMongoPlatzi

Observemos que la carpeta *myDockerData* tiene nueva informacion:

![](https://i.imgur.com/cTrq831.png)


### Recuperando los datos

Volvamos a crear un nuevo contenedor con el nombre *osito*, teniendo en cuenta de montar el directorio, que ya tiene nueva informacion:

    docker run -d --name osito --volume /home/carlos/Documents/curso_docker/myDockerData:/data/db mongo:5.0

Y efectivamente al hacer todo el procedimiento, y mirar con el querry:

    use platzi
    db.users.find()

Y estaran los datos tal y como los habia guardado 😊😊


Esta es una manera muy practica de compartir los datos de mi disco hacia el contenedor, sin embargo esto tiene un riesgo, al darle acceso irrestrico de una parte del disco al contenedor, pero no es la practica mas segura.  

## Volumenes

Son una evolucion de con respecto a los *bind mounts* con respecto a la seguridad en entornos productivos.

    sudo docker volume ls 

Veras la gran cantidad de volumenes que ya existina sin siquiera saberlo.

A continucion crearemos uno en particular:

    docker volume create dbdata

Y efectivamente aparece

![](https://i.imgur.com/Y1qgFRp.png)



Entonces docker ya reservo un espacio en el disko para ese volumen. De la misma forma que hizimos anteriormente, en un nuevo contenedor le montamos el nuevo volumen. La operacion se hara con *mount*.

- src: fuente: dbdata
- dst: destino: /data/db

Con *mount src* en caso de que el volumen no existies, Docker lo crearia en ese momento

    docker run --name myMongo -d --mount src=dbdata,dst=/data/db mongo:5.0

Efectivamente lo creo. Usemos *inspect* sobre el contenedor para obtener informacion mas especifica:

    docker inspect myMongo 

![](https://i.imgur.com/bj58gdO.png)


Se esta trabajando para que solo muestre informacion relevante, hasta ahora solo he podido hacer esto.

    docker inspect --format '{{.Config.Hostname}}' myMongo | less 

### Ingresando al contenedor

Y crear una base de datos como se hizo anteriormente. y una vez creada remover el contenedor

    docker rm -f myMongo

Sin embargo, al volver a mostrar los volumenes, el volumen *dbdata* sigue existiendo. 

Crea un nuevo contenedor, y montale el mismo volumen.

    sudo docker run --name osito -d --mount src=dbdata,dst=/data/db mongo:5.0

Verificando que la informacion este en el volumen ✅

## Insertar y extraer archivos de un contenedor

Aprenderemos como manualmente extraer o insertar archivos en un contenedor independientemente si se usa *bind mounts* o volumenes.

Crearemos un archivo llamado *prueba.txt*.

Y corramos un nuevo contenedor con un proceso:

    sudo docker run --name alwaysup --detach ubuntu tail -f /dev/null

Y abramos un terminal dentro del contenedor

    sudo docker exec -it alwaysup bash

Y dentro de ese contenedor crear un folder llamado *testing*

    mkdir testing

### Copiar dentro del contenedor en la carpèta testing el archivo prueba

Salgamos del bash del contenedor con exit, para volver a la terminal local

    sudo docker cp prueba.txt alwaysup:/testing/prueba.txt

Donde *alwaysup* es el nombre del contenedor

![](https://i.imgur.com/Exf6nzC.png)


Volvamos a ejecutar una terminal dentro del contenedor, y verificar que el archivo existe. ✅

### Copiando un archivo de un contenedor a mi entorno local

Con el mismo archivo. Cerremos el modo iteractivo con exit

    sudo docker cp alwaysup:/testing/prueba.txt ./local.txt

Y va a salir:

    Successfully copied 2.048kB to /home/carlos/Documents/curso_docker/local.txt ✅

Importante saber que el contenedor no necesariamente tiene que estar corriendo para realizar la operacion de copia


### Datos en Docker

![](https://i.imgur.com/9YPoMhY.png)

La imagen muestra mi maquina local que contiene un contenedor, donde este ultimo tiene dos formas de acceder al sistema de archivos:

- bind mount
- volume

La gran diferencia entre las dos es que la primera se vincula directamente el sitema de archivos con el contenedor y el acceso es irrestricto. 

Por otro lado Volume es casi lo mismo, solo que el area del *filesystem* que ese contenedor esta manejando, es manejada por Docker, y no es tan facil accederla. Aunque mejora la seguridad

Por otro lado esta el *temporaly file systmem mount*(tmpfs mount), que es comoda para escribir archivos temporales, pero que no nos interesa consevar una vez el contenedor deaparezca